<a href="https://colab.research.google.com/github/ahmedwalidahmad-debug/car-type-classification/blob/main/inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ======================================
# 1️⃣ Imports
# ======================================
import os, shutil
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.metrics import TopKCategoricalAccuracy


In [ ]:
import os
import random
import shutil
import scipy.io
import kagglehub

# ===============================
# 1️⃣ تحميل الداتاسيت
# ===============================
print("Downloading dataset...")
DATASET_PATH = kagglehub.dataset_download(
    "eduardo4jesus/stanford-cars-dataset"
)
print("Dataset path:", DATASET_PATH)

# ===============================
# 2️⃣ اكتشاف المسارات تلقائيًا
# ===============================
IMAGES_PATH = None
ANNOS_PATH  = None

for root, dirs, files in os.walk(DATASET_PATH):
    if "cars_train_annos.mat" in files:
        ANNOS_PATH = os.path.join(root, "cars_train_annos.mat")
    if os.path.basename(root) == "cars_train" and "cars_train" in root:
        # folder that contains images
        IMAGES_PATH = root

print("Images path:", IMAGES_PATH)
print("Annotations path:", ANNOS_PATH)

if IMAGES_PATH is None or ANNOS_PATH is None:
    raise FileNotFoundError("❌ Could not locate images or annotations. Dataset structure is unexpected.")

# ===============================
# 3️⃣ إعداد output
# ===============================
OUTPUT_PATH = "/kaggle/working/cars_100_classes"
os.makedirs(OUTPUT_PATH, exist_ok=True)

# ===============================
# 4️⃣ تحميل الـ annotations
# ===============================
annos = scipy.io.loadmat(ANNOS_PATH)
annotations = annos["annotations"][0]

print("Total images:", len(annotations))

# ===============================
# 5️⃣ تجميع الصور حسب class
# ===============================
class_dict = {}

for ann in annotations:
    img_name = ann[5][0]
    class_id = int(ann[4][0][0])

    class_dict.setdefault(class_id, []).append(img_name)

print("Total classes:", len(class_dict))

# ===============================
# 6️⃣ اختيار 100 class
# ===============================
random.seed(42)
selected_classes = random.sample(list(class_dict.keys()), 100)
print("Selected classes:", len(selected_classes))

# ===============================
# 7️⃣ نسخ الصور
# ===============================
missing = 0

for cls in selected_classes:
    class_folder = os.path.join(OUTPUT_PATH, f"class_{cls}")
    os.makedirs(class_folder, exist_ok=True)

    for img in class_dict[cls]:
        src = os.path.join(IMAGES_PATH, img)
        dst = os.path.join(class_folder, img)

        if os.path.exists(src):
            shutil.copy(src, dst)
        else:
            missing += 1

print(f" DONE! Dataset ready at: {OUTPUT_PATH}")
print(f" Missing images (ignored safely): {missing}")


Using Colab cache for faster access to the 'stanford-cars-dataset' dataset.
Dataset path: /kaggle/input/stanford-cars-dataset
Images path: /kaggle/input/stanford-cars-dataset/cars_train/cars_train
Annotations path: /kaggle/input/stanford-cars-dataset/car_devkit/devkit/cars_train_annos.mat
Total images: 8144
Total classes: 196
Selected classes: 100
🔥 DONE! Dataset ready at: /kaggle/working/cars_100_classes
⚠️ Missing images (ignored safely): 0


In [ ]:
# ======================================
# 2️⃣ Create 75 Classes Dataset
# ======================================
SRC_PATH = "/kaggle/working/cars_100_classes"
DST_PATH = "/kaggle/working/cars_75_classes"

os.makedirs(DST_PATH, exist_ok=True)

all_classes = [d for d in os.listdir(SRC_PATH) if d.startswith("class_")]
selected_classes = random.sample(all_classes, 75)

for cls in selected_classes:
    src_folder = os.path.join(SRC_PATH, cls)
    dst_folder = os.path.join(DST_PATH, cls)
    if not os.path.exists(dst_folder):
        shutil.copytree(src_folder, dst_folder)

print(" Created dataset with 75 classes at:", DST_PATH)


✅ Created dataset with 75 classes at: /kaggle/working/cars_75_classes


In [ ]:
# ======================================
# 3️⃣ Build DataFrame
# ======================================
DATA_PATH = "/kaggle/working/cars_75_classes"
assert os.path.exists(DATA_PATH), "Dataset not found"

data = []
for cls in sorted(os.listdir(DATA_PATH)):
    if not cls.startswith("class_"):
        continue
    cls_path = os.path.join(DATA_PATH, cls)
    for img in os.listdir(cls_path):
        if img.lower().endswith(('.jpg','.jpeg','.png')):
            data.append({"filename": os.path.join(cls_path, img), "label": cls})

df = pd.DataFrame(data)
print("Total images:", len(df))
print("Total classes:", df["label"].nunique())


Total images: 3139
Total classes: 75


In [ ]:
# ======================================
# 4️⃣ Train / Validation Split
# ======================================
train_df, val_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df["label"],
    random_state=42
)

IMG_SIZE = 224
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("NUM_CLASSES:", NUM_CLASSES)


Found 2511 validated image filenames belonging to 75 classes.
Found 628 validated image filenames belonging to 75 classes.
NUM_CLASSES: 75


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint


In [ ]:
IMG_SIZE = 299   # InceptionV3 prefers 299x299
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=inception_preprocess,
    rotation_range=20,
    width_shift_range=0.20,
    height_shift_range=0.20,
    zoom_range=0.25,
    shear_range=0.20,
    horizontal_flip=True,
    brightness_range=[0.7, 1.3]
)

val_datagen = ImageDataGenerator(
    preprocessing_function=inception_preprocess
)

train_gen = train_datagen.flow_from_dataframe(
    train_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True
)

val_gen = val_datagen.flow_from_dataframe(
    val_df,
    x_col="filename",
    y_col="label",
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False
)

NUM_CLASSES = len(train_gen.class_indices)
print("NUM_CLASSES:", NUM_CLASSES)


Found 2511 validated image filenames belonging to 75 classes.
Found 628 validated image filenames belonging to 75 classes.
NUM_CLASSES: 75


In [ ]:
def build_inception_model(num_classes):
    base_model = InceptionV3(
        weights="imagenet",
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )

    # في الأول نجمّد أغلب الطبقات
    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.5)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=outputs)
    return model

model = build_inception_model(NUM_CLASSES)
model.summary()


87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 149, 149,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 149, 149,  │         96 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 149, 149,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 147, 147,  │      9,216 │ activation[0][0]  │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │         96 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 147, 147,  │     18,432 │ activation_1[0][… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 147, 147,  │        192 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 147, 147,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 73, 73,    │          0 │ activation_2[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 73, 73,    │      5,120 │ max_pooling2d[0]… │
│                     │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 73, 73,    │        240 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 73, 73,    │          0 │ batch_normalizat… │
│ (Activation)        │ 80)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 71, 71,    │    138,240 │ activation_3[0][… │
│                     │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 71, 71,    │        576 │ conv2d_4[0][0]    │
│ (BatchNormalizatio… │ 192)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_4        │ (None, 71, 71,    │          0 │ batch_normalizat

 Total params: 21,956,459 (83.76 MB)

 Trainable params: 153,675 (600.29 KB)

 Non-trainable params: 21,802,784 (83.17 MB)

In [ ]:
warmup_epochs = 10

model.compile(
    optimizer=Adam(1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

early_stop_warmup = EarlyStopping(
    monitor="val_loss",
    patience=3,
    restore_best_weights=True
)

history_warmup = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=warmup_epochs,
    callbacks=[early_stop_warmup]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 116s 1s/step - accuracy: 0.0439 - loss: 4.4440 - val_accuracy: 0.2054 - val_loss: 3.2697
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 917ms/step - accuracy: 0.1682 - loss: 3.3731 - val_accuracy: 0.2818 - val_loss: 2.8534
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 917ms/step - accuracy: 0.2001 - loss: 3.0055 - val_accuracy: 0.3408 - val_loss: 2.6246
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 911ms/step - accuracy: 0.3000 - loss: 2.6828 - val_accuracy: 0.3487 - val_loss: 2.4688
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 911ms/step - accuracy: 0.3395 - loss: 2.4886 - val_accuracy: 0.3838 - val_loss: 2.3315
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 918ms/step - accuracy: 0.3634 - loss: 2.3804 - val_accuracy: 0.3838 - val_loss: 2.2650
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 915ms/step - accuracy: 0.3754 - loss: 2.2847 - val_accuracy: 0.3997 - val_loss: 2.1726
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 72s 916ms/step - accuracy: 0.4235 - loss: 2.1540 - val_accura

In [ ]:
import tensorflow as tf

# فتح آخر 150 طبقة
for layer in model.layers[-150:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

model.compile(
    optimizer=Adam(1e-5),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.3,
    patience=3,
    min_lr=1e-7,
    verbose=1
)

early_stop_ft = EarlyStopping(
    monitor="val_loss",
    patience=6,
    restore_best_weights=True,
    verbose=1
)

checkpoint = ModelCheckpoint(
    "best_inception_v3_75classes.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)

history_ft = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=30,
    callbacks=[reduce_lr, early_stop_ft, checkpoint]
)


Epoch 1/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.4987 - loss: 1.8274
Epoch 1: val_loss improved from inf to 1.59208, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 126s 1s/step - accuracy: 0.4989 - loss: 1.8263 - val_accuracy: 0.5717 - val_loss: 1.5921 - learning_rate: 1.0000e-05
Epoch 2/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 887ms/step - accuracy: 0.5974 - loss: 1.4481
Epoch 2: val_loss improved from 1.59208 to 1.50819, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 75s 950ms/step - accuracy: 0.5973 - loss: 1.4481 - val_accuracy: 0.5637 - val_loss: 1.5082 - learning_rate: 1.0000e-05
Epoch 3/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 896ms/step - accuracy: 0.6213 - loss: 1.2894
Epoch 3: val_loss improved from 1.50819 to 1.36113, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 957ms/step - accuracy: 0.6212 - loss: 1.2897 - val_accuracy: 0.6162 - val_loss: 1.3611 - learning_rate: 1.0000e-05
Epoch 4/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 879ms/step - accuracy: 0.6789 - loss: 1.1364
Epoch 4: val_loss improved from 1.36113 to 1.26467, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 936ms/step - accuracy: 0.6788 - loss: 1.1364 - val_accuracy: 0.6465 - val_loss: 1.2647 - learning_rate: 1.0000e-05
Epoch 5/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 874ms/step - accuracy: 0.6930 - loss: 1.0542
Epoch 5: val_loss improved from 1.26467 to 1.24220, saving model to best_inception_v3_75classes.h5


...

79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 937ms/step - accuracy: 0.8736 - loss: 0.4010 - val_accuracy: 0.7564 - val_loss: 0.8713 - learning_rate: 1.0000e-05
Epoch 18/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 883ms/step - accuracy: 0.8771 - loss: 0.3862
Epoch 18: val_loss did not improve from 0.87133
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 932ms/step - accuracy: 0.8771 - loss: 0.3862 - val_accuracy: 0.7627 - val_loss: 0.8981 - learning_rate: 1.0000e-05
Epoch 19/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 879ms/step - accuracy: 0.8831 - loss: 0.3833
Epoch 19: val_loss did not improve from 0.87133
79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 930ms/step - accuracy: 0.8831 - loss: 0.3829 - val_accuracy: 0.7596 - val_loss: 0.8901 - learning_rate: 1.0000e-05
Epoch 20/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 886ms/step - accuracy: 0.8744 - loss: 0.3762
Epoch 20: val_loss improved from 0.87133 to 0.81210, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 75s 944ms/step - accuracy: 0.8745 - loss: 0.3761 - val_accuracy: 0.7818 - val_loss: 0.8121 - learning_rate: 1.0000e-05
Epoch 21/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 880ms/step - accuracy: 0.8976 - loss: 0.3114
Epoch 21: val_loss did not improve from 0.81210
79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 928ms/step - accuracy: 0.8976 - loss: 0.3116 - val_accuracy: 0.7627 - val_loss: 0.8354 - learning_rate: 1.0000e-05
Epoch 22/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 882ms/step - accuracy: 0.9091 - loss: 0.3034
Epoch 22: val_loss did not improve from 0.81210
79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 927ms/step - accuracy: 0.9091 - loss: 0.3034 - val_accuracy: 0.7898 - val_loss: 0.8199 - learning_rate: 1.0000e-05
Epoch 23/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 881ms/step - accuracy: 0.9234 - loss: 0.2667
Epoch 23: ReduceLROnPlateau reducing learning rate to 2.9999999242136253e-06.

Epoch 23: val_loss did not improve from 0.81210
79/79 ━━━━━━━━━━━━━━━━━━━━ 73s 929ms/step - accuracy: 0.9233 - loss: 0.2670 - 

79/79 ━━━━━━━━━━━━━━━━━━━━ 78s 990ms/step - accuracy: 0.9246 - loss: 0.2585 - val_accuracy: 0.7787 - val_loss: 0.8020 - learning_rate: 3.0000e-06
Epoch 25/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 914ms/step - accuracy: 0.9171 - loss: 0.2755
Epoch 25: val_loss improved from 0.80196 to 0.77388, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 78s 980ms/step - accuracy: 0.9171 - loss: 0.2753 - val_accuracy: 0.7803 - val_loss: 0.7739 - learning_rate: 3.0000e-06
Epoch 26/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 906ms/step - accuracy: 0.9296 - loss: 0.2377
Epoch 26: val_loss improved from 0.77388 to 0.76886, saving model to best_inception_v3_75classes.h5


79/79 ━━━━━━━━━━━━━━━━━━━━ 77s 968ms/step - accuracy: 0.9295 - loss: 0.2378 - val_accuracy: 0.7962 - val_loss: 0.7689 - learning_rate: 3.0000e-06
Epoch 27/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 884ms/step - accuracy: 0.9276 - loss: 0.2420
Epoch 27: val_loss did not improve from 0.76886
79/79 ━━━━━━━━━━━━━━━━━━━━ 74s 931ms/step - accuracy: 0.9276 - loss: 0.2419 - val_accuracy: 0.7978 - val_loss: 0.7747 - learning_rate: 3.0000e-06
Epoch 28/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 908ms/step - accuracy: 0.9398 - loss: 0.1974
Epoch 28: val_loss did not improve from 0.76886
79/79 ━━━━━━━━━━━━━━━━━━━━ 76s 956ms/step - accuracy: 0.9399 - loss: 0.1973 - val_accuracy: 0.7818 - val_loss: 0.8051 - learning_rate: 3.0000e-06
Epoch 29/30
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 904ms/step - accuracy: 0.9191 - loss: 0.2333
Epoch 29: ReduceLROnPlateau reducing learning rate to 8.999999636216671e-07.

Epoch 29: val_loss did not improve from 0.76886
79/79 ━━━━━━━━━━━━━━━━━━━━ 75s 954ms/step - accuracy: 0.9192 - loss: 0.2332 - v

In [ ]:
val_loss, val_acc = model.evaluate(val_gen)
print("Final Val Loss:", val_loss)
print("Final Val Accuracy:", val_acc)


20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 175ms/step - accuracy: 0.7938 - loss: 0.7976
Final Val Loss: 0.7688599824905396
Final Val Accuracy: 0.7961783409118652
